# Chapter 6 Prediction


This note does **not** intend to give a detailed discussion of prediction methods. Rather, this chapter offers a way of approaching a prediction problem for beginners. 




## 6.1 General Prediction Framework

Consider a function $f$ indexed by a vector of parameters $\boldsymbol{\beta} \equiv \left(\beta_1,\ldots, \beta_p\right)$...\\

In particular for a vector of observations ${\bf x} \equiv \left(x_1,\ldots,x_k\right)$ we can get a prediction by
\[
{\rm prediction}\   \gets f\left({\bf x},\boldsymbol{\beta}\right)
\]
If we change $\boldsymbol{\beta}$ our predictions will change.

We want to find the best $\boldsymbol{\beta}$ for predicting our data.

To do this, we search for the vector $\boldsymbol{\beta}$ that minimizes
\[
\operatorname{avg}\left[L\left( {\rm outcome} ,  f\left({\bf x},\boldsymbol{\beta}\right) \right)\right]
\]
Often use some heuristic to get a **minimizer**

Key components:

 criteria 
 flexible modeling 
 model selection
     criteria
     over fitting 
     test train
     cross-validation 


## 6.2 Criteria


### 6.2.1 Continuous outcomes

With continuous outcome, often use
\[
L\left(  {\rm outcome},  {\rm prediction}\right) = \left( {\rm outcome} - {\rm prediction}\right)^2
\]

Does some combination of the features predict outcome effectively?

Combining features to predict outcome  

Suppose each observation is of the form
\[
\left( {\rm outcome},  {x_1, \ldots, x_k}\right)
\]
where $ {x_1, \ldots, x_k}$ is a set of $k$ numeric features.


Note $\{0,1\}$ indicators of group membership count as quantitative. 

What is a simple way to combine features to predict outcome?

<span style="color:blue">Multiple linear regression!</span>

Choose $\beta_0,\beta_1,\ldots, \beta_k$ to minimize

\[
\operatorname{avg}L\left(  {\rm outcome}, {\beta_0 + \beta_1 x_1 + ... + \beta_k x_k}\right)
\]
where $L\left( {\rm outcome},  {\rm prediction}\right)$ is some **cost** of making <span style="color:blue"> prediction </span>, when the truth is outcome.



Often we use 
	\[
	L\left( {\rm outcome} ,  {\rm prediction}  \right) = \left( {\rm outcome}  - {\rm prediction}\right)^2
	\]



Linear Regression
	Is the truth often
	\[
	y = \beta_0 + \beta_1 x_1 + \ldots + \beta_k x_k + \epsilon
	\]
    
When do I care? 
If I'm trying to make a statement like... After adjusting for $x_2,\ldots,x_k$, I find that $x_1$ is associated with $y$
It can also result in a not-very-predictive model!

The Inference Question 

We will discuss this later, because it is important. It's a bit of a pipe-dream to believe we can <span style="color:red"> completely</span> adjust for confounding without randomization. Often important to decrease the effect of confounding though. However, it is dangerous to assume that effects are **linear**.



In [ ]:

##-----------------------------------------------##
## Lasso: linear regression with CV ####
## 

cv.linear.fit <- cv.glmnet(as.matrix(covariates.expanded), outcome, family = "gaussian")
plot(cv.linear.fit)

## Report number of features included in the optimal model
sum(coef(cv.linear.fit, s = "lambda.min") != 0) 
## Find names of the selected features

coefs <- coef(cv.linear.fit, s = "lambda.min") 
linear.selected.variable<-which(coefs[,1]!=0) 
linear.selected.variable<-linear.selected.variable[linear.selected.variable!=1]-1;
linear.selected.mat =as.matrix(covariates.expanded[,linear.selected.variable]);

## Refit the model on the full data set with the selected variable:
linear.fit<-glmnet(linear.selected.mat, 
                   outcome, family = "gaussian",lambda=0)
plot(x=predict(linear.fit,newx=linear.selected.mat),y=outcome)
##-----------------------------------------------##














### 6.2.2 Binary and categorical outcomes 

What about binary outcomes and categorical outcomes?


You may have learned it from previous stat classes:

The <span style="color:red"> likelihood </span> of observing a collection of data, given a model 

We generally choose parameters in a model to maximize that likelihood, which is equivalent to maximizing the "log-likelihood" or  minimizing the "negative-log-likelihood"

In problems with a likelihood, generally our loss is the negative-log-likelihood.



For a binary outcome $y$, we will often model the probability $y = 1$ as $f(x)$...

Leading us to the negative-log-likelihood
\[
L( y,  f(x)) = - y \cdot \log\left(\frac{ f(x) }{1-f(x) }\right) + \operatorname{log}\left(1- f(x) \right)
\]
Our ML methods will try to choose $f$ to minimize the average of that loss over all observations.\\

Slightly more traditional to model $g(x) = \operatorname{log}\left(\frac{f(x)}{1-f(x)}\right)$ (<span style="color:red"> Why? </span>)

If we try to minimize the negative-log-likelihood,  and we model 
\[
\operatorname{log}\left(\frac{f(x)}{1-f(x)}\right) = x_1\beta_1 + \ldots + x_p\beta_p
\]

or equivalently

\[
    f(x) = \frac{e^{x_1\beta_1 + \ldots + x_p\beta_p}}{1+ e^{x_1\beta_1 + \ldots + x_p\beta_p}}
\]

This is known as the <span style="color:blue"> logistic</span> regression.


In [ ]:

##-----------------------------------------------##
## Lasso: logistic regression with CV####
##
cv.logistic.fit <- cv.glmnet(as.matrix(covariates.expanded), outcome, family = "binomial")
plot(cv.logistic.fit)

## Report number of features included in the optimal model
sum(coef(cv.logistic.fit, s = "lambda.min") != 0) 

## Find names of the selected features
coefs <- coef(cv.logistic.fit, s = "lambda.min") 

logistic.selected.variable<-which(coefs[,1]!=0) logistic.selected.variable<-logistic.selected.variable[logistic.selected.variable!=1]-1; 

## Refit the model on the full data set with the selected variable:

logistic.selected.mat =as.matrix(covariates.expanded[,logistic.selected.variable]);
logistic.fit<-glmnet(logistic.selected.mat, 
                     outcome, family = "binomial",lambda=0)
plot(x=predict(logistic.fit,newx=logistic.selected.mat),y=outcome)
##-----------------------------------------------##




### 6.2.3 Categorical Outcome

If we observe categorical $y\in\{1,\ldots, K\}$...
We consider $K$ functions $f_1,\ldots, f_k$  and set the prob $y = j$ as $\frac{f_j(x)}{\sum_{k=1}^K f_k(x)}$


Leading us to the negative-log-likelihood
\[
L( y,  f_1(x),\ldots, f_K(x)) = -\log\left( f_{ y } (x)\right) + \operatorname{log}\left(\sum_{k=1}^K  f_k(x)\right)
\]

Our ML methods will try to choose $f_1,\ldots, f_K$ to minimize the average of that loss over all observations.\\

Slightly more traditional to model $g(x) = \log(f(x))$.

Loss-based Estimation with bells+whistles

All of these losses can be combined with
- Penalties
- Feature Expansions/Dictionaries

There we use linear combinations of dictionary elements (e.g., meta-features) to minimize a penalized loss.


 Big Picture
There are lots of candidate methods for building predictive models

There is no best method **overall**, but there often is a best method **for your data**. There are two keys for a method to do well on your data:
    
- It does not <span style="color:orange"> completely</span> miss important structure
- It admits a proper amount of <span style="color:blue"> complexity</span> for the <span style="color:blue"> number of observations</span> in your data.



The "proper" amount of complexity

How can we tell if we are choosing a model that is...
- <span style="color:orange"> sufficiently complex </span> (to capture the signal)...
- but not <span style="color:red"> overly complex!</span> (such that we overfit our training data)?\\

The proper degree of complexity is **extremely** data-dependent
So <span style="color:blue"> data-resampling, split-sample-validation or CV</span>




## 6.3 Flexible models 
	
Returning to the **prediction** problem...

Sometimes we just want a black-box that makes good predictions. Other times we want to see how much info about response is contained in **any** combination of our assays. 

We can create a nonlinear model with even only one single feature $x$. 
- <span style="color:orange">Local smoothing:</span> For each $x$-value, estimate outcome with an average of observations with nearby $x$-values.
- <span style="color:orange"> Meta-feature construction:</span> Rather than just using $x$ as our one feature, we use e.g. $x, x^2, \ldots, x^5$ as features; and then choose $\beta_0,\ldots,\beta_5$ to minimize
	\[
	\operatorname{avg}\left(y - \left[\beta_0 + \beta_1 x + \beta_2 x^2 + \ldots + \beta_5 x^5\right]\right)^2
	\]
This is like doing span style="color:blue">multiple regression} on $5$ features...

But our additional features are user-defined.

Meta-feature expansions (in one dimension)

For continuous features various expansions may be used:
- Polynomial basis

\[
		\begin{pmatrix} \,\\x\\ \, \end{pmatrix}
		\rightarrow
		\begin{pmatrix}
		\, & \, & \, & \, \\ x, & x^2, & \cdots, &  x^m\\ \, & \, & \, & \,
		\end{pmatrix}
\]
        
- Spline basis

\[
		\begin{pmatrix} \,\\x\\ \, \end{pmatrix}
		\rightarrow
		\begin{pmatrix}
		\, & \, & \, & \, \\  x, &  \left(x - t_1\right)_{+}, & \cdots, & \left(x - t_m\right)_{+}\\ \, & \, & \, & \,
		\end{pmatrix}
\]

- Others include higher order spline, Fourier, wavelet, among others.




Meta Feature Expansion Example
	With enough data, and a rich enough basis, many functions can be estimated
	
<figure>
  <img src="../Figures/Ch6/sin.png" alt="sin()" style="width:70%">
  <figcaption> </figcaption>
</figure>


Same ideas can be employed with multiple features:

For <span style="color:orange">local smoothing</span> we now need a **distance** for multivariate ${\bf x}$ 
	
Often use Euclidean distance:
	\[
	D\big[\underbrace{\left( x_{1},x_{2} \right)}_{\textrm{observation $1$}}, \underbrace{\left( x_{1},x_{2} \right)}_{\textrm{observation $2$}}\big] \equiv \sqrt{\left( x_{1} - x_{1} \right)^2 + \left( x_{2}  -  x_{2} \right)^2}
	\]
	
Could include weights  (important if we haven't scaled our features) or an appropriate  kernel for interesting data-types.


In the  meta-feature approach, many ways to build meta-features: 
- Additive: e.g. $( x_1 , x_2 ) \rightarrow ( x_1,x_1^2,x_1^3, x_2,x_2^2,x_2^3)$
- Interacting: e.g. $( x_1 ,x_2 ) \rightarrow ( x_1,x_1^2 ,  x_2,x_2^2 , x_1 x_2 ,  x_1^2 x_2  , x_1 x_2^2 )$

In [ ]:

##-----------------------------------------------##
## Notes ####
### We analyzed the NOAH data set and built a prediction model in Homework #2. 
### What was your conclusion?

### Now let's try to build a prediction model on the iris data set  
### We will examine alternatives to the two major decisions we can make:
###   a) covariates: including them in their original form, or maybe transfer them?
###   b) model: fit a logistic regression, or maybe just a linear regression? 
### Let's try these out!
### Remember that we may not be able to build a good prediction model...
###  ... if there is really no information in the data set after all! 
##-----------------------------------------------##


##-----------------------------------------------##
## Instruction ####
### If you are comfortable with R, 
###     try to write your own code (you can use mine for refence)
### If you are struggling with R, 
###     move my code (at the end of the script) to the right place
###     and execute them. 
##-----------------------------------------------##



##-----------------------------------------------##
## Read data ####
data(iris)
plot(iris)
names(iris)
### Let's try to predict/classify the species using 
### Using Sepal.Length, Sepal.Width, Petal.Length, Petal.Width 
### Yes, just 4 variables instead of the 56k variables in NOAH
### We will only consider TWO species: setosa and versicolor here
keep.ind = which(iris$Species != 'virginica');
outcome = iris$Species[keep.ind]=='setosa';
covariates=iris[keep.ind,1:4];
##-----------------------------------------------##

##-----------------------------------------------##
## Meta-feature generation ####
### Task: construct meta-features based on the 4  variables in covariates 
### Several options are: 
### (Orthogonal) Polynomial bases (?poly), splines (?spline),Fourier bases (?create.fourier.basis) (NOTE: this requires the "fda" package), among others 
### The sample script will use the polynomial bases
### You should try out a different type of bases if you are fluent with R 

### Create polynomial bases for one variable and visualize them  
M=5; 
tmp <- covariates[,1];
tmp.expanded<-poly(tmp, degree = M,raw=F); # Set raw=T to see the raw polynomials 


### For visualization, you can do it manually, or use the melt() function in reshape2 
#library(reshape2)
library(reshape2)
tmp.plot <- data.frame(tmp.expanded);
names(tmp.plot)= paste('x', 1:M, sep='');
tmp.plot.long <- melt(tmp.plot, id="x1")  
ggplot(data=tmp.plot.long,
       aes(x=x1, y=value, colour=variable)) +
  geom_line()

### Now create a data frame that contains the expanded features:
### Hint: just recycle the code in visualization! (not very elegant but working)
for(i in 1:dim(covariates)[2]){
  tmp <- covariates[,i];
  tmp.name=names(covariates)[i];
  tmp.expanded<-poly(tmp, degree = M,raw=F);
  tmp.plot <- data.frame(tmp.expanded);
  names(tmp.plot)= paste(tmp.name,'Poly',1:M, sep='');
  if(i==1){
    covariates.expanded= tmp.plot;
  }else{
    covariates.expanded= cbind(covariates.expanded,tmp.plot);
  }
}



### Choosing an approach

With $3+$ features often better to use meta-feature approach, than local-averaging...

Any guesses why local-averaging has trouble?

With more features, local neighborhood becomes  sparse

Generally, more  uniformative features than informative... So, distance becomes dominated by uninformative component. 



Sometimes have many candidate features:
- Biomolecular 'omics data: $1,000-10^9$ features (eg. genes).
- Unstructured text data: $1,000+$ meta-features (eg. word-counts).
- Image data: $10,000+$ features (eg. voxels)

In many of these scenarios we
- Have more features than observations
- Believe most features are not informative for response
- But do not know which those are!


Number of obs determines complexity that can be supported:

$100$ obs cannot support even linear model with $1000$ features


Too much flexibility $\longrightarrow$ over-adapts to training data, e.g.	

Univariate smoothing with tiny bandwidth $\longrightarrow$ interpolates points
Restricting the number of features allowed in the model is one way of limiting its complexity




## Tree-Based Methods

The main steps of tree-based methods are as follows:

- *stratify* (or segment) the predictor space into small and simple regions
- for each observation, determine which segment it belongs to; the stratification of space can be captured by a tree (hence the name)
- predict the outcome by the mean (regression), mode (classification), or hazard (survival) of outcomes in that segment



Decision Trees: the general idea

- The *regression* tree in the above example is likely an over-simplification of the true relationship 
- However, it is very easy to interpret, and has a nice graphical representation: you can easily explain it to a non-statistician, and do not need a computer (or even a calculator) to get an estimate!

The general steps for building a regression (or classification) tree is quite simple:
- Divide the predictor space, i.e. the set of possible values for $X_1, X_2, \ldots, X_p$, into {\color{red} $J$ distinct and non-overlapping regions}, $R_1, R_2, \ldots, R_J$.
- Use the mean (regression) or mode (classification) of the response values for the training observations in region $R_j$ as the predicted response for that region.




    
The main question: <span style="color:red"> how should we construct the regions</span> $R_1, \ldots, R_J$, and <span style="color:red"> how many regions should we use</span>? 


Partitioning the Predictor Space
Suppose $J$ is known. How to construct the regions $R_1, \ldots, R_J$?
- Divide the predictor space into high-dimensional <span style="color:red"> rectangles</span> or boxes (in theory any shape can be used, but this is simpler)
- Find boxes $R_1, \ldots, R_J$ that minimize the RSS
\[
\sum_{j=1}^J \sum_{i \in R_j} (y_i - {\hat{y}_{R_j}})^2
\]
- Unfortunately, it is not possible to consider every possible partition of the space into $J$ boxes

We consider instead a {\it greedy} approach called {<span style="color:blue"> recursive binary splitting</span>
- top-down: similar to hierarchical clustering
- best split is determined at each given step, instead of the best global step (which is computationally difficult)

                                                    
                                                    
- Select a predictor $X_j$ and a <span style="color:red"> cut point</span> $t$ such that splitting the predictor space into regions

\[
 R_1(j, t) = \{ X \mid X_j < t\}, \hspace{1cm} R_2(j, t) = \{ X \mid X_j \ge t\}
\]
 gives the largest decrease in RSS.
 - We consider *all predictors* and *all cut points* for each predictor!
 - We find the value of $j$ and $t$ that minimizes
\[
 \sum_{i: x_i \in R_1(j,t)} (y_i - \hat{y}_{R_1})^2 + \sum_{i: x_i \in R_2(j,t)} (y_i - \hat{y}_{R_2})^2
\]

Finding *optimal* $j$ and $s$ can actually be done very quickly (as long as $p$ is not too large)

Repeat the above process, splitting a subspace (rather than the whole space) in each step
- Previously used variables can be considered again




Tree Pruning
<span style="color:red"> complexity</span> of the regression tree is determined by the number of regions $J$

- A tree with large $J$ may work well in training data, but would be very bad on test data
- A smaller tree with fewer splits might have lower variance and better interpretation at the cost of a little bias
- One option is to consider a split only if the drop in RSS is larger than some (high) threshold
- However, this may not be a good strategy since a so-so split at step $j$ may be followed by a great one at step $j+1$
- Instead, we <span style="color:blue"> first grow a large tree</span>, e.g. until no region has $>5$ observations, and then <span style="color:red"> prune</span> it to obtain a <span style="color:red"> subtree</span>
- And, again, we (basically) use <span style="color:red"> cross validation</span> to select $J$


Our goal is to select a subtree that leads to the {\color{red} lowest test error}

We can use CV, however, it will be too computationally expensive to estimate the CV error for every possible subtree!
- Instead we use a strategy called <span style="color:red"> cost complexity pruning</span> a.k.a. <span style="color:red"> weakest link pruning</span>
- Rather than considering every possible subtree, we consider a sequence of trees indexed by a nonnegative tuning parameter $\alpha$
- For each value of $\alpha$, there exists a subtree $T \subset T_0$ such that 
\[
\sum_{k=1}^{|T|}\sum_{x_i \in R_k} (y_i - \hat{y}_{R_k})^2 + \alpha |T|
\]
 is as small as possible ($|T|$ is the number of leaves of the tree) 
- $\alpha$ controls the tradeoff between complexity and fit (sound familiar?)
- We can select $\alpha$ using validation set or CV approach



Extensions of Trees 
Trees can be seen as a loss-based ML method

As such, trees can extend naturally to binary/survival data choose splits greedily to minimize loss
                                                    
                                                    
Trees are often used as <span style="color:blue"> base learners</span>
<span style="color:red"> model aggregation/ensembling</span>, e.g.
- Boosting
- Bagging
- Stacking

Trees in high dimensions
Some `R` functions are better equipped to deal with high dimensional data than others...
Rarely will it hurt your method to prescreen out features that are extremely marginally uncorrelated with outcome.
Remember to be careful how you do this, with validation!

Another approach is to screen out features with very low variance
Alternatively, some packages handle big data better (eg. `Ranger` / `XGBoost`)


In [ ]:

##---------------------------------------##
## Decistion tree ####
### 
library(tree)

fit_tree <- tree(Y ~ Xmat)

cv_fit_tree <- cv.tree(fit_tree)

plot(cv_fit_tree)


## Model selection


### Selecting an appropriate model

Model selection has many meanings 
- It can mean selecting a $\lambda$-value in the lasso, known as <span style="color:blue"> tuning parameter selection</span>
- It can mean selecting covariates and how they are to be included in the model, known as feature selection, and sometimes feature generation 
- It can mean choosing between a regression tree (random forest), neural network (deep learning), or logistic regression





Here we use feature selection to fit a model with only a small number of those features.

One method is **best subset selection**:

Choose $\beta_0,\beta_1,\ldots, \beta_k$, with no more than $C$ of those $k$ nonzero to minimize
\[
	\operatorname{avg}L\left({\rm outcome} , {\beta_0 + \beta_1 x_1 + ... \beta_k x_k}\right)
\]
	   
Here we use a constraint on the allowable $\beta_0,\ldots,\beta_k$ to select our features.



In practice *best subset selection* is computationally intractable. Use a slightly different approach, called the **Lasso**

We choose $\beta_0,\beta_1,\ldots, \beta_k$, to minimize
	\[
	\operatorname{avg}\left[L\left( {\rm outcome} ,  {\beta_0 + \beta_1 x_1 + ... +\beta_k x_k}\right)\right] + \lambda P\left(\beta_1,\ldots,\beta_k\right)
	\]
	with $ P\left(\beta_1,\ldots,\beta_k\right) = |\beta_1| + \ldots + |\beta_k| $, a function that penalizes complexity in $\beta$\\
	  
Minimizing $\beta$-values, will give sparse model (meaning: many $\beta$-values exactly equal to $0$) 

To modulate the degree of sparsity, we change $\lambda$.

Adding penalties for  complexity  is a general strategy. One could consider choosing $\beta_0,\beta_1,\ldots, \beta_k$, to minimize
	\[
	\operatorname{avg}L\left( {\rm outcome} ,  {\beta_0 + \beta_1 x_1 + ... \beta_k x_k}\right) + \lambda P\left(\beta_1,\ldots,\beta_k\right)
	\]
 with a variety of different $P$
 


 Another popular choice is $P(\beta_1,\ldots,\beta_k) = \beta_1^2 + \ldots + \beta_k^2$.
 
 **Ridge Regression** or **Tikhonov Regularization**   Does not result in a *sparse model*. In all these cases increasing $\lambda$ results in a *less complex* predictive model.
 

In some cases we may: 	
- First grow our feature set by creating meta-features
- Then select the useful meta-features by using a \textbf{lasso} fit


For high dimensional problems, expand each variable
	\vspace{3mm}
	\begin{align*}
	&\,\,\,\,\,\,\,\,\begin{pmatrix}
	& & & \\  x_1 , &  x_2 , &  \cdots &  x_k \\ & & &
	\end{pmatrix}\\
	&\rightarrow
	\begin{pmatrix}
	& & & & & & & & & \\ 
	 x_1 , &  \cdots , &  x_1^m , & 
	 x_2 , &  \cdots , &  x_2^m , &
	\cdots, &
	 x_k , &  \cdots , &  x_k^m \\
	& & & & & & & & &
	\end{pmatrix}
	\end{align*}
    
	and use the **lasso** on this expanded problem.
	- $m$ should be small ($\sim 5$ish) 
	- **Spline** basis generally outperforms **polynomial**
	


    
    More General Machine Learning Methods
	There are many other methods that can be used for prediction:

Just to name some examples:
- Regression Trees (boosted trees, random forests)
- Neural Networks (deep learning)
- Support Vector Machines

All of these work in a very similar way, that is actually not that complicated




## Validation 

Split Sample Validation - Lasso

For choosing $\lambda$ in the lasso...
- Split your data into a <span style="color:blue"> training</span> set and a <span style="color:orange"> validation</span> set
- Choose candidate $\lambda$-values ($\lambda_1,\ldots,\lambda_m$)
- For each candidate $\lambda$-value, fit your Lasso model on the <span style="color:blue"> training</span> data to get a set of coefficients\\
- Evaluate each of the $m$ sets of coefficients on your <span style="color:orange"> validation</span> data.

<span style="color:red"> Final Step (why?):</span> Refit the Lasso model to **all the data**, using the best $\lambda$-value. 


More Generally

Suppose we are deciding between a <span style="color:blue"> neural network </span>, a <span style="color:red"> regression tree with depth $3$</span>, and a <span style="color:orange"> regression tree with depth $5$</span>.\\
- Split your data into a training set and a validation set
- For each of our $3$ procedures, build a predictive model on the training data
- Evaluate each of the $3$ predictive models on the validation data.

<span style="color:red"> Final Step:</span> Refit a predictive model to the full data, using the **best** procedure.